In [ ]:
import networkx as nx
import pandas as pd
from tqdm import tqdm_notebook,tqdm
import numpy as np
parent_path='../Data/New_Data_15-06-2020/'

In [ ]:
annotated_df=pd.read_pickle(parent_path+'Fearspeech_data_final.pkl')
whatsapp_data=pd.read_csv(parent_path+'Data_text_spam_removed_v02.csv')
temp=whatsapp_data[['group_id_anonymized','phone_num_anonymized','message_text','timestamp']]
duplicateDFRow = temp[temp.duplicated()]
whatsapp_data=whatsapp_data.drop(list(duplicateDFRow.index))


In [ ]:
whatsapp_data=whatsapp_data.sort_values('timestamp')

In [ ]:
df_user_matched=pd.read_csv(parent_path+'final_user_selection_matched.csv')

In [ ]:
len(df_user_matched[df_user_matched['annotation']=='Non fear speech'])

In [ ]:
dict_users_ids={}
dict_users_names={}
dict_users_groups={}  
count=0
for user in tqdm(whatsapp_data['phone_num_anonymized'].unique()):
    if user in annotated_df.phone_num_anonymized.unique():
        dict_users_ids[count]=user
        dict_users_names[count]='fear speech'
    else:
        dict_users_ids[count]=user
        dict_users_names[count]='normal'
        
    dict_users_groups[count]=set(whatsapp_data[whatsapp_data['phone_num_anonymized']==user].group_id_anonymized)
    count+=1

In [ ]:
tuples_user_info=[]
for user1 in tqdm(dict_users_ids):
    for user2 in dict_users_ids:
        if(user1==user2):
            continue
        temp_user1_groups= dict_users_groups[user1]
        temp_user2_groups= dict_users_groups[user2]
        common_groups=len(temp_user1_groups.intersection(temp_user2_groups))
        if(common_groups>0):
            tuples_user_info.append([user1,user2,common_groups])

In [ ]:
import networkx as nx

In [ ]:
G = nx.Graph()

In [ ]:
G.add_weighted_edges_from(tuples_user_info)

In [ ]:
import json

def convert(o):
    if isinstance(o, np.generic): return o.item()  
    raise TypeError

#json.dumps({'value': numpy.int64(42)}, default=convert)


with open("Networks/user_dict.json", 'w') as fp:
    json.dump(dict_users_ids, fp,ensure_ascii=False,default=convert)
with open("Networks/user_labels_dict.json", 'w') as fp:
    json.dump(dict_users_names, fp,ensure_ascii=False,default=convert)
nx.write_weighted_edgelist(G, "Networks/users_network.edgelist.gz")

### Load precomputed

In [ ]:
import json

with open("Networks/user_dict.json", 'r') as fp:
    dict_users_ids=json.load(fp)
with open("Networks/user_labels_dict.json", 'r') as fp:
    dict_users_names=json.load(fp)

    
G=nx.read_weighted_edgelist("Networks/users_network.edgelist.gz")
    

In [ ]:
k_cores=nx.algorithms.core.core_number(G)

In [ ]:
closeness_centrality=nx.algorithms.centrality.closeness_centrality(G)

In [ ]:
all_fear_speech_index=[]
all_normal_index=[]

count_fearspeech=0
count_normal=0

for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if(row['one_fear_speech']==1):
        count_fearspeech+=1
        all_fear_speech_index+=row['repeated']
    elif(row['one_fear_speech']==0):
        count_normal+=1
        all_normal_index+=row['repeated']

data_fear_speech=whatsapp_data[whatsapp_data['orig_index'].isin(all_fear_speech_index)]
data_normal=whatsapp_data[whatsapp_data['orig_index'].isin(all_normal_index)]


In [ ]:
len(data_fear_speech.phone_num_anonymized.unique())

In [ ]:
list_messages=[]
for user in data_fear_speech.phone_num_anonymized.unique():
    temp_user=data_fear_speech[data_fear_speech['phone_num_anonymized']==user]
    list_messages.append(len(temp_user))
    



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['figure.figsize'] = (5, 5)

#     'weight' : 'normal',
#     'size'   : 15}
# matplotlib.rc('font', **font)
#data_timeline.plot()
sns.set(context='paper',style='white',font_scale=1.5)

font = {
    'weight' : 'normal',
    'size'   : 11}
ax = sns.histplot(x=list_messages,bins=100)

plt.xlabel('amount of fear speech')
plt.ylabel('number of users')

plt.yscale('log')
plt.savefig('../Fear_speech_results/fear_speech_per_users.pdf',bbox_inches='tight',dpi=400)

plt.show()

In [ ]:


%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams['figure.figsize'] = (5, 5)

#     'weight' : 'normal',
#     'size'   : 15}
# matplotlib.rc('font', **font)
#data_timeline.plot()
sns.set(context='paper',style='white',font_scale=1.5)

font = {
    'weight' : 'normal',
    'size'   : 11}


list_messages=[]
for user in data_fear_speech.group_id_anonymized.unique():
    temp_user=data_fear_speech[data_fear_speech['group_id_anonymized']==user]
    list_messages.append(len(temp_user))
    
ax = sns.histplot(x=list_messages,bins=100)

plt.xlabel('amount of fear speech')
plt.ylabel('number of groups')

plt.yscale('log')
plt.savefig('../Fear_speech_results/fear_speech_per_groups.pdf',bbox_inches='tight',dpi=400)

plt.show()

In [ ]:
list_cores=[]
for user in data_fear_speech.phone_num_anonymized.unique():
    try:
        list_cores.append(k_cores[int(user)])
    except:
        list_cores.append(0)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (5, 5)

#     'weight' : 'normal',
#     'size'   : 15}
# matplotlib.rc('font', **font)
#data_timeline.plot()
sns.set(context='paper',style='white',font_scale=1.5)

font = {
    'weight' : 'normal',
    'size'   : 11}
ax = sns.histplot(x=list_cores,bins=50)

plt.xlabel('core number')
plt.ylabel('number of fear speech users')

#plt.yscale('log')
plt.savefig('../Fear_speech_results/kcore_fear_speech_users.pdf',bbox_inches='tight',dpi=400)

plt.show()

In [ ]:
user_nfs=df_user_matched[df_user_matched['annotation']=='Non fear speech'].user_id

In [ ]:
len(user_nfs)

In [ ]:
list_cores=[]
for user in user_nfs:
    try:
        list_cores.append(k_cores[int(user)])
    except:
        list_cores.append(0)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (5, 5)

#     'weight' : 'normal',
#     'size'   : 15}
# matplotlib.rc('font', **font)
#data_timeline.plot()
sns.set(context='paper',style='white',font_scale=1.5)

font = {
    'weight' : 'normal',
    'size'   : 11}
ax = sns.histplot(x=list_cores,bins=50,color='#DE8F05')

plt.xlabel('core number')
plt.ylabel('number of non fear speech users')

#plt.yscale('log')
plt.savefig('../Fear_speech_results/kcore_non_fear_speech_users.pdf',bbox_inches='tight',dpi=400)

plt.show()

In [ ]:
admin_data=pd.read_csv(parent_path+'group_membership.csv',sep='\t')
admin_data[admin_data['is_admin']!=0]

In [ ]:
num_spreader=[]
for index,row in tqdm(df_user_matched.iterrows(),total=len(df_user_matched)):
    try:
        temp=admin_data[admin_data['user_id']==row['user_id']].group_id.nunique()
    except KeyError:
        temp=0
    num_spreader.append([row['user_id'],temp,row['annotation']])
    

In [ ]:
import seaborn as sns

matplotlib.rcParams['figure.figsize'] = (5, 5)

#     'weight' : 'normal',
#     'size'   : 15}
# matplotlib.rc('font', **font)
#data_timeline.plot()
sns.set(context='paper',style='white',font_scale=1.5)

font = {
    'weight' : 'normal',
    'size'   : 11}


df_num_users=pd.DataFrame(num_spreader,columns=['user_id','number of groups','label'])
ax1 = sns.barplot(x="label", y="number of groups", data=df_num_users)
plt.savefig('../Fear_speech_results/admins_in_fear_speech_users.pdf',bbox_inches='tight',dpi=400)
plt.show()


In [ ]:
len(df_num_users[(df_num_users['label']=='Fear speech') & (df_num_users['number of groups']>0)])/len(df_num_users[(df_num_users['label']=='Fear speech')])

In [ ]:
all_fear_speech_index=[]
all_normal_index=[]

count_fearspeech=0
count_normal=0

for index,row in tqdm(annotated_df.iterrows(),total=len(annotated_df)):
    if(row['one_fear_speech']==1):
        count_fearspeech+=1
        all_fear_speech_index+=row['repeated']
    elif(row['one_fear_speech']==0):
        count_normal+=1
        all_normal_index+=row['repeated']

data_fear_speech=whatsapp_data[whatsapp_data['orig_index'].isin(all_fear_speech_index)]
data_normal=whatsapp_data[whatsapp_data['orig_index'].isin(all_normal_index)]


In [ ]:
temp=data_fear_speech.sample(1)
whatsapp_data[(whatsapp_data['group_id_anonymized']==temp.iloc[0]['group_id_anonymized']) & (whatsapp_data['timestamp'].between(temp.iloc[0]['timestamp'],temp.iloc[0]['timestamp']+(2*24*60*60*1000) , inclusive=True))]

In [ ]:
user_actions=pd.read_csv(parent_path+'user_actions.csv',sep='\t')
user_actions=user_actions.sort_values('timestamp')

In [ ]:
user_actions.head()

In [ ]:
count_no_left=0
time_to_leave=[]
for index,row in tqdm_notebook(data_fear_speech.iterrows(),total=len(data_fear_speech)):
    temp_action=user_actions[(user_actions['timestamp']>int(row['timestamp']/1000))&(user_actions['group_id']==row['group_id_anonymized'])&(user_actions['user_id']==row['phone_num_anonymized'])]
    flag=0
    if(len(temp_action)==0):
        count_no_left+=1
        continue
    for index1,row1 in temp_action.iterrows():
        if(row1['action'] in ['removed']):
            time_shift=(row1['timestamp']-int(row['timestamp']/1000))/(24*60*60)
            time_to_leave.append(time_shift)
            flag=1
            break
    if(flag==0):
        count_no_left+=1
        
           
    
        
        

In [ ]:
len(time_to_leave)/len(data_fear_speech)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.hist(time_to_leave, bins = 100)
plt.xlabel('days when removed from the group')
plt.ylabel('number of cases')
plt.show()

In [ ]:
user_actions.action.unique()

### Groups